In [89]:
import pandas as pd
import requests
import json
import datetime

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### What are your parameters ?

In [51]:
after = '2021-12-31'
before = '2022-01-02'
subreddit = 'france'

In [26]:
def get_pushshift_data(after, before, subreddit):
    URL = 'https://api.pushshift.io/reddit/submission/search/?subreddit='+str(subreddit)+'&after='+str(after)+'&before='+str(before)
    print(URL)
    r = requests.get(URL)
    data = json.loads(r.text, strict = False)
    return data['data']

In [79]:

def collect_clean_data(subpost, columns):
    clean_data = list()
    title = subpost['title']
    url = subpost['url']
    try:
        flair = subpost['link_flair_text']
    except KeyError:
        flair = 'NaN'
    try:
        body = subpost['selftext']
    except KeyError:
        body = ''
    author = subpost['author']
    postId = subpost['id']
    score = subpost['score']
    created = datetime.datetime.fromtimestamp(subpost['created_utc'])
    num_com = subpost['num_comments']
    permalink = subpost['permalink']    
    return pd.Series([postId,title,body,url,author,score,created,num_com,permalink,flair], index = columns)

In [86]:
def loop_between_dates(after, before, subreddit):
    #our data will be collected in this dataframe
    columns = ['postId','title','body','url','author','score','created','num_com','permalink','flair']
    sub_clean_data = pd.DataFrame(columns=columns)
    #tracks number of posts saved
    subpost_count = 0
    
    data = get_pushshift_data(after, before, subreddit)
    while len(data) > 0:
        for subpost in data:
            subpost_list = collect_clean_data(subpost, columns)
            print(subpost_list)
            sub_clean_data = sub_clean_data.append(subpost_list, ignore_index=True)
            subpost_count += 1
        print(len(data))
        print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']

        #New request
        data = get_pushshift_data(after, before, subreddit)

In [85]:
pd.Series(['d', 'e'], index = ['postId','title'])

postId    d
title     e
dtype: object

In [84]:
df = pd.DataFrame(columns=['postId','title','body','url','author','score','created','num_com','permalink','flair'])
df = df.append(pd.Series(['a', 'b'], index = ['postId','title']), ignore_index=True)
df = df.append(pd.Series(['d', 'e'], index = ['postId','title']), ignore_index=True)
df

C:\Users\matth\AppData\Local\Temp\ipykernel_11856\2309960398.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(['a', 'b'], index = ['postId','title']), ignore_index=True)
C:\Users\matth\AppData\Local\Temp\ipykernel_11856\2309960398.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(['d', 'e'], index = ['postId','title']), ignore_index=True)


,postId,title,body,url,author,score,created,num_com,permalink,flair
0,a,b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
loop_between_dates(after, before, subreddit)
sub_clean_data


https://api.pushshift.io/reddit/submission/search/?subreddit=france&after=2021-12-31&before=2022-01-02
postId                                                  rsg4ft
title        L’Université de Paris sommée de changer de nom...
body                                                          
url          https://www.lemonde.fr/societe/article/2021/12...
author                                                 kanetix
score                                                        1
created                                    2021-12-31 01:01:42
num_com                                                      0
permalink    /r/france/comments/rsg4ft/luniversité_de_paris...
flair                                                      NaN
dtype: object
postId                                                  rsgl2u
title        Maxime Vachier-Lagrave, champion du monde en B...
body                                                          
url          /r/chess/comments/rs9hfr/congrats_to_the_world...
a

KeyboardInterrupt: 